## Installing  Librries

In [11]:
# pip install transformers
# pip install datasets
# pip install ipywidgets
# pip install torch torchvision torchaudio

## Loading Libraries

In [1]:
import os
import sys
import datetime 
import pandas as pd 
from transformers import BertTokenizer, AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, pipeline
import numpy as np
from datasets import load_metric
import nltk
from datasets import load_dataset
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


## Get Current working directory

In [2]:
my_dir = os.getcwd() + '/'

## Loading Datasets

In [10]:
dataset = load_dataset('csv', data_files={'train': './train_df.csv', 'test': './test_df.csv'})

Using custom data configuration default-3ab95468b09c363b


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\Bilal\.cache\huggingface\datasets\csv\default-3ab95468b09c363b\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## Bert Tokenizer

<p>Here we load BERT tokenizer and saving it in directory folder save_tokenizer</p>

In [4]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer.save_pretrained(my_dir+"save_tokenizer")

('C:\\Users\\Bilal\\Desktop\\NLP\\Fiver\\bert/save_tokenizer\\tokenizer_config.json',
 'C:\\Users\\Bilal\\Desktop\\NLP\\Fiver\\bert/save_tokenizer\\special_tokens_map.json',
 'C:\\Users\\Bilal\\Desktop\\NLP\\Fiver\\bert/save_tokenizer\\vocab.txt',
 'C:\\Users\\Bilal\\Desktop\\NLP\\Fiver\\bert/save_tokenizer\\added_tokens.json',
 'C:\\Users\\Bilal\\Desktop\\NLP\\Fiver\\bert/save_tokenizer\\tokenizer.json')

In [5]:
tokenizer = AutoTokenizer.from_pretrained('./save_tokenizer/')

In [11]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [12]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets['train']
small_eval_dataset = tokenized_datasets['test']

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Loading Model

In [16]:
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

## Saving Model

In [13]:
model.save_pretrained(my_dir+"save_model")

## Loading Modeel From Save Dir

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("./save_model/", num_labels=2)

In [17]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Setting TrainingArguments

In [18]:
training_args = TrainingArguments(
         overwrite_output_dir = 'True',
         output_dir="./fined_tuned_model/",
         evaluation_strategy="epoch",
         num_train_epochs=3, 
         save_total_limit = 2,
         save_strategy = "no",
         load_best_model_at_end=False)

## Setting Trainer Arguments

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset ,
    compute_metrics=compute_metrics,
)

## Training Start

In [22]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Bilal\Anaconda3\envs\tensorflow\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2016
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 756


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.192468,0.928000
2,0.318500,0.024874,0.994000
3,0.318500,0.006498,0.998000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


Training complete

TrainOutput(global_step=756, training_loss=0.21630305461782626, metrics={'train_runtime': 36266.4035, 'train_samples_per_second': 0.167, 'train_steps_per_second': 0.021, 'total_flos': 1591295662817280.0, 'train_loss': 0.21630305461782626, 'epoch': 3.0})

## Saving the model

In [23]:
trainer.save_model()

Saving model checkpoint to ./fined_tuned_model/
Configuration saved in ./fined_tuned_model/config.json
Model weights saved in ./fined_tuned_model/pytorch_model.bin


## Calling Prediction On Eval Datasat

In [24]:
predictions = trainer.predict(small_eval_dataset)
print(list(np.argmax(predictions.predictions, axis=-1)))

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 500
  Batch size = 8


[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Loading the save tokeniizer and model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./save_tokenizer/')
model = AutoModelForSequenceClassification.from_pretrained("./fined_tuned_model/", num_labels=2)
#calling prediction

print("\n\nLabel 0 = test1\nLabel 1 = test2\n")
for i in range(20):
    print("Enter Text")
    text=str(input())
    print("\nSentence: {}".format(text))
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    result=classifier(text)
    print("\n",result)
    label=result[0]['label']
    accuracy=result[0]['score']
    print("Label: ",label)
    print("Accuracy:",accuracy)

Didn't find file ./save_tokenizer/tokenizer.json. We won't load it.
Didn't find file ./save_tokenizer/added_tokens.json. We won't load it.
loading file ./save_tokenizer/vocab.txt
loading file None
loading file None
loading file ./save_tokenizer/special_tokens_map.json
loading file ./save_tokenizer/tokenizer_config.json
loading configuration file ./fined_tuned_model/config.json
Model config BertConfig {
  "_name_or_path": "./fined_tuned_model/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",



Label 0 = test1
Label 1 = test2

Enter Text
We were greeted in an inquisitive but friendly manner and handed a coconut fresh from the tree . 

Sentence: We were greeted in an inquisitive but friendly manner and handed a coconut fresh from the tree . 

 [{'label': 'LABEL_0', 'score': 0.9998779296875}]
Label:  LABEL_0
Accuracy: 0.9998779296875
Enter Text
She was never taken out of the bed , until the ambulance came and took her out the bed this morning .

Sentence: She was never taken out of the bed , until the ambulance came and took her out the bed this morning .

 [{'label': 'LABEL_1', 'score': 0.9997208714485168}]
Label:  LABEL_1
Accuracy: 0.9997208714485168
Enter Text
Not long after takeoff from Madrid , the airplane discovers a problem with the landing gear .

Sentence: Not long after takeoff from Madrid , the airplane discovers a problem with the landing gear .

 [{'label': 'LABEL_1', 'score': 0.9997456669807434}]
Label:  LABEL_1
Accuracy: 0.9997456669807434
Enter Text
Do Not Sk